In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

In [2]:
train_df = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train.csv')
test_df = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test.csv')  
submission = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/sample_submission.csv')  

## 영어로 번역된 부분 삭제

In [4]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

## 특수기호 제거

In [6]:
df_combined = pd.concat([train_df, test_df], axis=0)

In [7]:
# 기호를 추출하는 함수
def extract_symbols(text):
    return re.findall(r'[^\w\s,]', text)  # 알파벳, 숫자, 공백, 쉼표를 제외한 모든 기호

# '키워드' 열에서 기호 추출
symbols = df_combined['키워드'].apply(extract_symbols)

# 기호 리스트 출력
unique_symbols = set([symbol for sublist in symbols for symbol in sublist])
print(unique_symbols)

{'‧', '•', ':', '·', '○', ';', '?', '/', '%', '․', '&', '▲', '■', '→', '.', ')', '▷', '⋅', '’', '㈜', '㎡', '=', '“', '⋯', '!', '~', '・', '∙', '(', '-', '˙', '*'}


In [8]:
# 제거할 기호 리스트
symbols_to_remove = {'‧', '•', ':', '·', '○', ';', '?', '/', '%', '․', '&', '▲', '■', '→', '.', ')', '▷', '⋅', '’', '㈜', '㎡', '=', '“', '⋯', '!', '~', '・', '∙', '(', '-', '˙', '*'}

# 기호를 제거하는 함수
def remove_symbols(text):
    pattern = '[' + re.escape(''.join(symbols_to_remove)) + ']'  # 제거할 기호 패턴 생성
    return re.sub(pattern, '', text)

# '키워드' 열에서 기호 제거
df_combined['키워드'] = df_combined['키워드'].apply(remove_symbols)

삭제가 제대로 되었는지 확인

In [9]:
# '키워드' 열에서 기호 추출
symbols = df_combined['키워드'].apply(extract_symbols)

# 기호 리스트 출력
unique_symbols = set([symbol for sublist in symbols for symbol in sublist])
print(unique_symbols)

set()
